In [1]:
import numpy as np
import math
from numpy import linalg as LA

In [2]:
def RosenbrockFunction(x):
    return (1-x[0])**2+100*(x[1]-x[0]**2)**2

def Derivative_Rosenbrock (x):
    dx = 2*(200*x[0]**3-200*x[0]*x[1]+x[0]-1)
    dy = 200*(x[1] - x[0]**2)
    return np.array([dx, dy])

def Hessian_Rosenbrock(x):
    dxx = 1200*x[0]**2-400*x[1]+2
    dxy = -400*x[0]
    dyx=-400*x[0]
    dyy=200
    return np.array([dxx,dxy,dyx,dyy]).reshape(2,2)

def RosenbrockFunction_alpha(x,alpha,dfx):
    x0=x[0]-alpha*dfx[0]
    x1=x[1]-alpha*dfx[1]
    return (1-x0)**2+100*(x1-x0**2)**2

def HimmelblauFunction(x):
    return (x[0]**2+x[1]-11)**2+(x[0]+x[1]**2-7)**2

def Derivative_Himmelblau (x):
    dx = 2*(2*x[0]*(x[0]**2+x[1]-11)+x[0]+x[1]**2-7)
    dy = 2*(x[0]**2+2*x[1]*(x[0]+x[1]**2-7)+x[1]-11)
    return np.array([dx, dy])

def Hessian_Himmelblau(x):
    dxx=12*x[0]**2+4*x[1]-42
    dxy=4*(x[0]+x[1])
    dyx=4*(x[0]+x[1])
    dyy=4*x[0]+12*x[1]**2-26
    return np.array([dxx,dxy,dyx,dyy]).reshape(2,2)

def HimmelblauFunction_alpha(x,alpha,dfx):
    x0=x[0]-alpha*dfx[0]
    x1=x[1]-alpha*dfx[1]
    return (x0**2+x1-11)**2+(x0+x1**2-7)**2

In [3]:
# 1-dimension search algorithm
def GoldenSectionSearchMethod(x,fx,dfx,lp,rp,epsilon):
    
    '''
     Parameters:
     fx: target  function
     lp: left point
     rp: right point
     epsilon: precision
     return: minimum_point, minimum_value
    '''
    G = np.zeros((1000, 4))
    delta = rp - lp
    t = (math.sqrt(5)-1)/2
    x1 = lp + (1-t)*delta
    x2 = lp + t*delta
    fxx1 = fx(x,x1,dfx)
    fxx2 = fx(x,x2,dfx)
    k = 0
    G[k] = [lp,x1,x2,rp]
    while abs(rp-lp) > epsilon: 
        #print("iteration",k)
       # print(G[k])
        if fxx1 < fxx2:
            rp=x2
            x2 = x1
            fxx2 = fxx1
            x1 = lp + (1-t)*(rp-lp)
            fxx1 = fx(x,x1,dfx)
        else:
            lp = x1
            x1 = x2
            fxx1 = fxx2
            x2=lp+t*(rp-lp)
            fxx2=fx(x,x2,dfx)
        k = k + 1
        G[k] = lp, x1, x2, rp
    min_point = (x1+x2)/2
    min_value = fx(x,min_point,dfx)
    return min_value, min_point,G[k]

def find_step_length(f, fd, x, alpha, direction, c2):
  g = lambda alpha: f(x+alpha*direction)
  gd = lambda alpha: np.dot(fd(x + alpha*direction), direction)
  return interpolation(g, gd, alpha, c2)

def wolf1(f, fd, alpha):
  c1 = 1e-3
  return f(alpha) <= f(0) + c1*alpha*fd(alpha)

def wolf_strong(f, fd, alpha, c2):
  return abs(fd(alpha)) <= -c2*fd(0)

def simple_backtracking(f, fd, alpha, c2):
  rate = 0.5
  while not (wolf1(f, fd, alpha) or wolf_strong(f, fd, alpha, c2)):
    alpha = rate*alpha
  return alpha

def interpolation(f, fd, alpha, c2):
  lo = 0
  hi = 1
    
  for i in range(0, 20):
    if wolf1(f, fd, alpha):
      if wolf_strong(f, fd, alpha, c2):
        return alpha
    
    half = (lo+hi)/2.0
    alpha = - (fd(lo)*hi*hi) / (2*(f(hi)-f(lo)-fd(lo)*hi))
    
    if alpha < lo or alpha > hi: # quadratic interpolation failed. reduce by half instead
      alpha = half
    if fd(alpha) > 0:
      hi = alpha
    elif fd(alpha) <= 0:
      lo = alpha
  return alpha

In [4]:
# gradient Descent fixed learning rate
def GradientDescentMethod(func,dfunc,alpha=0.003):
    '''
    GradientDescentMethod fixed learning rate
    alpha: learning rate(default:0.003)
    func: target function
    '''

    # initialize a point
    x = np.array([-1,1])
    # set number of epochs
    epoch = 10000
    alpha=0.002
    xi=np.zeros_like(x)
    for i in range(epoch):
        
        
        dfxi = dfunc(x)
        # set the new point
        xi = x - alpha*dfxi
        if LA.norm(xi-x,2)<=1e-5 or LA.norm(func(xi)-func(x))<=1e-5 or LA.norm(dfunc(xi))<=1e-5:
            break
        x = xi

        print("epoch:",i)
        print(xi,func(xi))

In [5]:
GradientDescentMethod(RosenbrockFunction,Derivative_Rosenbrock)

epoch: 0
[-0.992  1.   ] 3.9934596096000003
epoch: 1
[-0.99667881  0.9936256 ] 3.9867328710613266
epoch: 2
[-0.98889697  0.99352282] 3.980064635794433
epoch: 3
[-0.99328725  0.98728058] 3.9732377460774115
epoch: 4
[-0.98583937  0.98701617] 3.9664706028526866
epoch: 5
[-0.98983406  0.98096141] 3.9595811845574853
epoch: 6
[-0.982817    0.98048543] 3.9527514834982993
epoch: 7
[-0.98633058  0.97466296] 3.9458385709541695
epoch: 8
[-0.97981737  0.97393698] 3.9389836486829144
epoch: 9
[-0.98278967  0.96837902] 3.9320816290215346
epoch: 10
[-0.97682683  0.96737763] 3.9252339429655914
epoch: 11
[-0.97922463  0.96210284] 3.9183682519883183
epoch: 12
[-0.97383175  0.96081406] 3.9115513436683913
epoch: 13
[-0.97564808  0.95582774] 3.904736569514539
epoch: 14
[-0.97081961  0.95425232] 3.8979634618938186
epoch: 15
[-0.97207105  0.94954768] 3.891203789210361
epoch: 16
[-0.96777986  0.94769745] 3.884477680765346
epoch: 17
[-0.96850231  0.94325761] 3.877769046855109
epoch: 18
[-0.96470445  0.94115326]

epoch: 413
[0.1719328  0.02705529] 0.6863230927636422
epoch: 414
[0.17490043 0.02805753] 0.6814307207626898
epoch: 415
[0.17784646 0.02907058] 0.6765911755558808
epoch: 416
[0.18077102 0.03009409] 0.6718038609565821
epoch: 417
[0.18367424 0.03112772] 0.6670681805433556
epoch: 418
[0.18655625 0.03217112] 0.6623835381535443
epoch: 419
[0.18941719 0.03322397] 0.65774933834968
epoch: 420
[0.19225722 0.03428593] 0.6531649868594457
epoch: 421
[0.19507646 0.03535669] 0.6486298909899582
epoch: 422
[0.19787508 0.03643595] 0.6441434600171633
epoch: 423
[0.20065323 0.03752339] 0.6397051055511479
epoch: 424
[0.20341105 0.03861872] 0.6353142418781875
epoch: 425
[0.20614871 0.03972165] 0.6309702862803598
epoch: 426
[0.20886636 0.04083191] 0.6266726593335514
epoch: 427
[0.21156416 0.04194921] 0.6224207851846913
epoch: 428
[0.21424228 0.04307328] 0.6182140918090359
epoch: 429
[0.21690087 0.04420387] 0.6140520112483291
epoch: 430
[0.2195401  0.04534072] 0.6099339798306449
epoch: 431
[0.22216013 0.04648

epoch: 692
[0.57425543 0.32765592] 0.18170507618291112
epoch: 693
[0.57498751 0.32850127] 0.18108055800066114
epoch: 694
[0.57571727 0.32934502] 0.18045908605833744
epoch: 695
[0.57644473 0.33018716] 0.17984063942254658
epoch: 696
[0.5771699  0.33102771] 0.179225197351552
epoch: 697
[0.57789278 0.33186666] 0.17861273929307328
epoch: 698
[0.5786134  0.33270402] 0.17800324488211494
epoch: 699
[0.57933176 0.3335398 ] 0.17739669393882632
epoch: 700
[0.58004788 0.334374  ] 0.17679306646638918
epoch: 701
[0.58076177 0.33520662] 0.17619234264893527
epoch: 702
[0.58147344 0.33603766] 0.17559450284949174
epoch: 703
[0.58218291 0.33686714] 0.17499952760795462
epoch: 704
[0.58289018 0.33769506] 0.17440739763908958
epoch: 705
[0.58359526 0.33852142] 0.17381809383056004
epoch: 706
[0.58429818 0.33934622] 0.17323159724098175
epoch: 707
[0.58499893 0.34016948] 0.17264788909800338
epoch: 708
[0.58569754 0.34099119] 0.1720669507964135
epoch: 709
[0.58639401 0.34181136] 0.171488763896272
epoch: 710
[0.5

[0.72810408 0.52886517] 0.07408877754048078
epoch: 1000
[0.72845169 0.52937332] 0.07389940431862176
epoch: 1001
[0.72879863 0.52988074] 0.07371063736683359
epoch: 1002
[0.7291449  0.53038742] 0.07352247417694376
epoch: 1003
[0.7294905  0.53089336] 0.07333491225411251
epoch: 1004
[0.72983545 0.53139858] 0.07314794911674438
epoch: 1005
[0.73017973 0.53190306] 0.07296158229640046
epoch: 1006
[0.73052335 0.53240681] 0.07277580933771137
epoch: 1007
[0.73086632 0.53290983] 0.07259062779829098
epoch: 1008
[0.73120863 0.53341213] 0.07240603524865069
epoch: 1009
[0.73155029 0.5339137 ] 0.07222202927211446
epoch: 1010
[0.73189129 0.53441455] 0.0720386074647345
epoch: 1011
[0.73223165 0.53491468] 0.07185576743520779
epoch: 1012
[0.73257136 0.53541408] 0.07167350680479284
epoch: 1013
[0.73291043 0.53591277] 0.07149182320722773
epoch: 1014
[0.73324885 0.53641074] 0.07131071428864812
epoch: 1015
[0.73358663 0.536908  ] 0.07113017770750649
epoch: 1016
[0.73392377 0.53740454] 0.0709502111344917
epoch:

epoch: 1370
[0.8234402  0.67726983] 0.03123481698903415
epoch: 1371
[0.82363002 0.67758341] 0.031167678554711965
epoch: 1372
[0.82381958 0.67789661] 0.03110070694572825
epoch: 1373
[0.82400886 0.67820945] 0.0310339016651471
epoch: 1374
[0.82419788 0.67852191] 0.0309672622178398
epoch: 1375
[0.82438663 0.678834  ] 0.030900788110476984
epoch: 1376
[0.82457512 0.67914573] 0.030834478851520696
epoch: 1377
[0.82476334 0.67945709] 0.030768333951216375
epoch: 1378
[0.8249513  0.67976808] 0.030702352921585012
epoch: 1379
[0.82513899 0.68007871] 0.03063653527641538
epoch: 1380
[0.82532642 0.68038897] 0.03057088053125625
epoch: 1381
[0.82551359 0.68069886] 0.03050538820340855
epoch: 1382
[0.82570049 0.68100839] 0.03044005781191773
epoch: 1383
[0.82588713 0.68131756] 0.03037488887756611
epoch: 1384
[0.82607352 0.68162636] 0.030309880922865226
epoch: 1385
[0.82625964 0.6819348 ] 0.030245033472048283
epoch: 1386
[0.8264455  0.68224287] 0.03018034605106254
epoch: 1387
[0.8266311  0.68255059] 0.03011

epoch: 1741
[0.87911425 0.77232202] 0.014640388631875351
epoch: 1742
[0.87923219 0.77252995] 0.014611829934155178
epoch: 1743
[0.87935    0.77273767] 0.01458333271154488
epoch: 1744
[0.87946767 0.77294517] 0.014554896813301873
epoch: 1745
[0.8795852  0.77315246] 0.014526522089115444
epoch: 1746
[0.87970259 0.77335952] 0.014498208389105344
epoch: 1747
[0.87981985 0.77356637] 0.014469955563820256
epoch: 1748
[0.87993696 0.77377301] 0.014441763464236437
epoch: 1749
[0.88005394 0.77397943] 0.01441363194175619
epoch: 1750
[0.88017078 0.77418563] 0.014385560848206469
epoch: 1751
[0.88028749 0.77439162] 0.014357550035837429
epoch: 1752
[0.88040406 0.7745974 ] 0.01432959935732101
epoch: 1753
[0.88052049 0.77480296] 0.014301708665749504
epoch: 1754
[0.88063678 0.77500831] 0.014273877814634146
epoch: 1755
[0.88075294 0.77521344] 0.014246106657903723
epoch: 1756
[0.88086896 0.77541836] 0.014218395049903102
epoch: 1757
[0.88098485 0.77562307] 0.014190742845391947
epoch: 1758
[0.8811006  0.77582756

[0.91272611 0.832701  ] 0.007630270377294371
epoch: 2086
[0.91280654 0.83284818] 0.007616212228087733
epoch: 2087
[0.91288688 0.83299522] 0.0076021818780830355
epoch: 2088
[0.91296714 0.83314211] 0.007588179266721474
epoch: 2089
[0.91304731 0.83328887] 0.0075742043335933285
epoch: 2090
[0.9131274  0.83343548] 0.007560257018437544
epoch: 2091
[0.9132074  0.83358194] 0.0075463372611412685
epoch: 2092
[0.91328732 0.83372827] 0.0075324450017395
epoch: 2093
[0.91336715 0.83387445] 0.007518580180414588
epoch: 2094
[0.9134469 0.8340205] 0.007504742737495927
epoch: 2095
[0.91352657 0.8341664 ] 0.00749093261345943
epoch: 2096
[0.91360615 0.83431216] 0.007477149748927198
epoch: 2097
[0.91368565 0.83445778] 0.007463394084667028
epoch: 2098
[0.91376507 0.83460325] 0.007449665561592086
epoch: 2099
[0.9138444  0.83474859] 0.007435964120760435
epoch: 2100
[0.91392365 0.83489379] 0.007422289703374665
epoch: 2101
[0.91400281 0.83503884] 0.007408642250781447
epoch: 2102
[0.91408189 0.83518376] 0.0073950

In [6]:
GradientDescentMethod(HimmelblauFunction,Derivative_Himmelblau)

epoch: 0
[-1.044  1.092] 124.70179826739198
epoch: 1
[-1.09024233  1.18712727] 119.01296703362092
epoch: 2
[-1.13873857  1.28507356] 112.9514860257079
epoch: 3
[-1.18947831  1.38543987] 106.54843888060553
epoch: 4
[-1.24242511  1.48773273] 99.8490995298954
epoch: 5
[-1.29751239  1.59136455] 92.91317108647195
epoch: 6
[-1.35463937  1.69565958] 85.8139146397502
epoch: 7
[-1.41366744  1.79986603] 78.63599614980699
epoch: 8
[-1.47441707  1.90317495] 71.47203689786471
epoch: 9
[-1.53666587  2.00474548] 64.41806042158798
epoch: 10
[-1.60014808  2.10373522] 57.5682476750603
epoch: 11
[-1.66455592  2.19933382] 51.00958670038788
epoch: 12
[-1.72954308  2.29079686] 44.81707701585053
epoch: 13
[-1.79473052  2.37747704] 39.05008697710318
epoch: 14
[-1.85971459  2.45884951] 33.750268656283176
epoch: 15
[-1.92407716  2.53452912] 28.941156731959982
epoch: 16
[-1.98739736  2.60427815] 24.629291245509677
epoch: 17
[-2.04926416  2.66800427] 20.806484536506176
epoch: 18
[-2.10928896  2.72575002] 17.45274

In [7]:
def GradientDescentMethod_optAlpha(func,dfunc,Opti_func):
    '''
    func: target function
    dfunc: derivative of target function
    Opti_func: Optimization function of Alpha
    '''

    # initialize a point
    x = np.array([0,0])
    # set number of epochs
    epoch = 1000
    xi=np.zeros_like(x)
    for i in range(epoch):
        
        
        dfxi = dfunc(x)
        #Wolfe condition line search
#         alpha = find_step_length(func, dfunc, x, 1.0, -dfunc(x), c2=0.9)
        
        _,alpha,_=GoldenSectionSearchMethod(x,Opti_func,dfxi,1e-3,1,1e-5)
        print("Learning Rate:",alpha)
        # set the new point
        xi = x - np.dot(alpha,dfxi.T)
        if LA.norm(xi-x,2)<=1e-5 or LA.norm(func(xi)-func(x))<=1e-5 or LA.norm(np.array(dfunc(xi)),2)<=1e-5:
            break
        x = xi
        print("epoch:",i)
        print("Min Point:"+str(xi)+"\t Min Value:"+str(func(xi)))

In [8]:
GradientDescentMethod_optAlpha(RosenbrockFunction,Derivative_Rosenbrock,RosenbrockFunction_alpha)

Learning Rate: 0.08063280868005282
epoch: 0
Min Point:[0.16126562 0.        ]	 Min Value:0.771109685558644
Learning Rate: 0.00499704337946184
epoch: 1
Min Point:[0.16126502 0.02599122]	 Min Value:0.703476388155829
Learning Rate: 0.030046461365503867
epoch: 2
Min Point:[0.21163762 0.02608248]	 Min Value:0.6565141846288627
Learning Rate: 0.0049933630487846315
epoch: 3
Min Point:[0.21160266 0.04476565]	 Min Value:0.6215703781938366
Learning Rate: 0.02144609711681416
epoch: 4
Min Point:[0.24540054 0.04480869]	 Min Value:0.5931755884377097
Learning Rate: 0.0049933630487846315
epoch: 5
Min Point:[0.24538197 0.06020096]	 Min Value:0.5694483851657332
Learning Rate: 0.01729656428224151
epoch: 6
Min Point:[0.27146731 0.06024021]	 Min Value:0.5488616606123692
Learning Rate: 0.0049933630487846315
epoch: 7
Min Point:[0.27144786 0.07367664]	 Min Value:0.5307882229336394
Learning Rate: 0.01472833162236471
epoch: 8
Min Point:[0.2928969  0.07369815]	 Min Value:0.514612674699051
Learning Rate: 0.0049933

epoch: 103
Min Point:[0.58906704 0.34698712]	 Min Value:0.16886591770160084
Learning Rate: 0.003569369944600976
epoch: 104
Min Point:[0.59198977 0.3469963 ]	 Min Value:0.16766645908821304
Learning Rate: 0.0049621827869279645
epoch: 105
Min Point:[0.59197861 0.35042576]	 Min Value:0.16648147434547558
Learning Rate: 0.0035322347826187463
epoch: 106
Min Point:[0.59485026 0.35043488]	 Min Value:0.16531045392086374
Learning Rate: 0.004971818017730735
epoch: 107
Min Point:[0.5948426 0.3538276]	 Min Value:0.16415252948921225
Learning Rate: 0.00349737419008487
epoch: 108
Min Point:[0.59766815 0.35383468]	 Min Value:0.1630083203710049
Learning Rate: 0.004971818017730735
epoch: 109
Min Point:[0.59766019 0.35718821]	 Min Value:0.16187732988245834
Learning Rate: 0.003466193928228202
epoch: 110
Min Point:[0.6004415  0.35719479]	 Min Value:0.16075935188500118
Learning Rate: 0.004968137687053526
epoch: 111
Min Point:[0.60043196 0.36050874]	 Min Value:0.1596546292840222
Learning Rate: 0.00343133333569

Learning Rate: 0.0020889712168295485
epoch: 332
Min Point:[0.77307383 0.59617439]	 Min Value:0.051711211295568826
Learning Rate: 0.0049814532485335065
epoch: 333
Min Point:[0.77307218 0.5976377 ]	 Min Value:0.05149623601760312
Learning Rate: 0.002083016316703986
epoch: 334
Min Point:[0.7740157  0.59763891]	 Min Value:0.05128247184837881
Learning Rate: 0.004971818017730735
epoch: 335
Min Point:[0.77401326 0.59909207]	 Min Value:0.05107000726130564
Learning Rate: 0.0020793359860267775
epoch: 336
Min Point:[0.7749502  0.59909393]	 Min Value:0.050858790689051595
Learning Rate: 0.004968137687053526
epoch: 337
Min Point:[0.77494734 0.60053854]	 Min Value:0.050648701120216806
Learning Rate: 0.002073381085901215
epoch: 338
Min Point:[0.77587747 0.60054055]	 Min Value:0.05043979696878819
Learning Rate: 0.0049814532485335065
epoch: 339
Min Point:[0.77587595 0.60198049]	 Min Value:0.05023159052740401
Learning Rate: 0.0020674261857756525
epoch: 340
Min Point:[0.77680075 0.60198173]	 Min Value:0.05

epoch: 444
Min Point:[0.81709188 0.66650761]	 Min Value:0.03358341666260336
Learning Rate: 0.004723244731096893
epoch: 445
Min Point:[0.81707294 0.66757651]	 Min Value:0.033462410849359185
Learning Rate: 0.0019064387845862505
epoch: 446
Min Point:[0.81775068 0.66758859]	 Min Value:0.03334195961780646
Learning Rate: 0.004701699700042996
epoch: 447
Min Point:[0.8177303  0.66864891]	 Min Value:0.033222360389152314
Learning Rate: 0.0019064387845862505
epoch: 448
Min Point:[0.81840411 0.66866184]	 Min Value:0.033103279985844544
Learning Rate: 0.004682429238437452
epoch: 449
Min Point:[0.81838266 0.66971393]	 Min Value:0.03298498893283731
Learning Rate: 0.0019064387845862505
epoch: 450
Min Point:[0.81905252 0.66972775]	 Min Value:0.03286726874397962
Learning Rate: 0.004666839107509119
epoch: 451
Min Point:[0.8190301  0.67077246]	 Min Value:0.03275024957855173
Learning Rate: 0.0019064387845862505
epoch: 452
Min Point:[0.81969648 0.67078688]	 Min Value:0.03263377876072118
Learning Rate: 0.0046

epoch: 590
Min Point:[0.85614599 0.73211867]	 Min Value:0.02076919476733742
Learning Rate: 0.004208233457452519
epoch: 591
Min Point:[0.85610685 0.73284862]	 Min Value:0.020705733238004766
Learning Rate: 0.0018128979990162473
epoch: 592
Min Point:[0.85658492 0.73287411]	 Min Value:0.020642467722643755
Learning Rate: 0.004220143257703646
epoch: 593
Min Point:[0.85654663 0.73360303]	 Min Value:0.020579346259267082
Learning Rate: 0.0018092176683390387
epoch: 594
Min Point:[0.85702287 0.73362803]	 Min Value:0.020516448512116675
Learning Rate: 0.004229778488506416
epoch: 595
Min Point:[0.85698514 0.7343557 ]	 Min Value:0.020453709702582014
Learning Rate: 0.0018032627682134763
epoch: 596
Min Point:[0.857459   0.73438016]	 Min Value:0.020391171454008896
Learning Rate: 0.004260958750363085
epoch: 597
Min Point:[0.85742307 0.73510944]	 Min Value:0.02032860253252252
Learning Rate: 0.0017995824375362673
epoch: 598
Min Point:[0.85789618 0.73513279]	 Min Value:0.020266267640111867
Learning Rate: 0.

Min Point:[0.88917418 0.78997296]	 Min Value:0.012325628072952692
Learning Rate: 0.0037110868841848353
epoch: 775
Min Point:[0.88912855 0.79046116]	 Min Value:0.012293260421136926
Learning Rate: 0.0017684021756795999
epoch: 776
Min Point:[0.88946507 0.79049243]	 Min Value:0.01226096225232561
Learning Rate: 0.0037110868841848353
epoch: 777
Min Point:[0.88941975 0.79097909]	 Min Value:0.012228773331034294
Learning Rate: 0.0017684021756795999
epoch: 778
Min Point:[0.88975523 0.79101036]	 Min Value:0.012196682706246907
Learning Rate: 0.0037014516533820643
epoch: 779
Min Point:[0.88970979 0.79149452]	 Min Value:0.012164722662469589
Learning Rate: 0.0017684021756795999
epoch: 780
Min Point:[0.89004386 0.79152599]	 Min Value:0.01213287339622596
Learning Rate: 0.0037014516533820643
epoch: 781
Min Point:[0.88999856 0.79200872]	 Min Value:0.012101104426575747
Learning Rate: 0.0017684021756795999
epoch: 782
Min Point:[0.89033176 0.7920401 ]	 Min Value:0.012069443944684333
Learning Rate: 0.0036895

epoch: 919
Min Point:[0.90764824 0.82373993]	 Min Value:0.008529576385944706
Learning Rate: 0.0017372219138229322
epoch: 920
Min Point:[0.90791525 0.8237696 ]	 Min Value:0.00850881494338529
Learning Rate: 0.00349737419008487
epoch: 921
Min Point:[0.90787286 0.82414767]	 Min Value:0.008488140488533438
Learning Rate: 0.0017372219138229322
epoch: 922
Min Point:[0.90813904 0.82417736]	 Min Value:0.008467504778712642
Learning Rate: 0.0034914192899593074
epoch: 923
Min Point:[0.9080967  0.82455384]	 Min Value:0.008446952725190538
Learning Rate: 0.0017372219138229322
epoch: 924
Min Point:[0.90836188 0.82458364]	 Min Value:0.008426453485415696
Learning Rate: 0.0034914192899593074
epoch: 925
Min Point:[0.90831969 0.82495909]	 Min Value:0.008406011204542327
Learning Rate: 0.0017372219138229322
epoch: 926
Min Point:[0.90858422 0.82498882]	 Min Value:0.008385624155045155
Learning Rate: 0.0034817840591565355
epoch: 927
Min Point:[0.90854196 0.82536239]	 Min Value:0.00836531400142988
Learning Rate: 

In [9]:
GradientDescentMethod_optAlpha(HimmelblauFunction,Derivative_Himmelblau,HimmelblauFunction_alpha)

Learning Rate: 0.12733746690826853
epoch: 0
Min Point:[1.78272454 2.80142427]	 Min Value:32.12570410247361
Learning Rate: 0.03988828628613393
epoch: 1
Min Point:[3.00087482 2.02607906]	 Min Value:0.012190196706611872
Learning Rate: 0.015536412005979572
epoch: 2
Min Point:[2.99174256 2.01177514]	 Min Value:0.002940965552835866
Learning Rate: 0.022625197265041183
epoch: 3
Min Point:[3.00018644 2.00638383]	 Min Value:0.0007199951917170293
Learning Rate: 0.015652903583832325
epoch: 4
Min Point:[2.9979706 2.0029126]	 Min Value:0.00017846815769435599
Learning Rate: 0.022599971903310076
epoch: 5
Min Point:[3.00004487 2.00158759]	 Min Value:4.437903468326065e-05
Learning Rate: 0.015684083845688992
epoch: 6
Min Point:[2.99949471 2.00072597]	 Min Value:1.1071095980819737e-05
Learning Rate: 0.022580701441704536


In [10]:
rho=0.5
c=0.5

In [11]:
def ConjugateGradientMethod(func,dfunc,Opti_func):
    '''
    func: target function
    Opti_func: Optimization function of Alpha
    '''

    # initialize a point
    x = np.array([-1,1])
    dfx_next=dfunc(x)
    # set number of epochs
    epoch = 1000
    x_prev=None
    dfx=None
    alpha=1
    for i in range(epoch):
#         #Wolfe condition line search
#         alpha = find_step_length(func, dfunc, x, 1, -dfx_next, c2=0.1)
        # Golden Search
        _,alpha,_=GoldenSectionSearchMethod(x,Opti_func,dfx_next,1e-3,1,1e-5)
#         # Backtracking line search 
#         while func(x + alpha*(-dfunc(x))) > func(x) + c*alpha*np.dot(dfunc(x).T, (-dfunc(x))):
#             print("alpha = %.4f | f(x + alpha*(-dfunc(x))) = %.4f" %
#                   (alpha, func(x + alpha*(-dfunc(x)))))
#             alpha = rho * alpha
        print("Learning Rate:",alpha)
        # set the new point
        x_prev=x
        x=x-np.dot(alpha,dfx_next)
        # update gradient
        dfx=dfx_next
        dfx_next=dfunc(x)
        if LA.norm(x-x_prev)<=1e-5 or LA.norm(x-x_prev)<=1e-5 or LA.norm(dfx_next)<=1e-5:
            break
        # Polak-Reiber method
        # Bk=(np.dot(dfx_next,dfx_next-dfx))/(np.dot(dfx,dfx))
        # Fletcher–Reeves
        Bk=(np.dot(dfx_next,dfx_next))/(np.dot(dfx,dfx))
        dfx_next=dfx_next+Bk*dfx_next

        print("epoch:",i)
        print("Min Point:"+str(x)+"\t Min Value:"+str(func(x)))

In [12]:
ConjugateGradientMethod(RosenbrockFunction,Derivative_Rosenbrock,RosenbrockFunction_alpha)

Learning Rate: 0.5000005788572839
epoch: 0
Min Point:[1.00000232 1.        ]	 Min Value:2.1498510102125504e-09
Learning Rate: 0.0010048176154013855


In [13]:
ConjugateGradientMethod(HimmelblauFunction,Derivative_Himmelblau,HimmelblauFunction_alpha)

Learning Rate: 0.049522353412789585
epoch: 0
Min Point:[-2.08949178  3.27802826]	 Min Value:14.004969654491738
Learning Rate: 0.013603362528277105
epoch: 1
Min Point:[-2.71441736  2.9791008 ]	 Min Value:1.1307486907137063
Learning Rate: 0.01316034700914884
epoch: 2
Min Point:[-2.78992253  3.1369355 ]	 Min Value:0.008848223425266765
Learning Rate: 0.01449902879733641
epoch: 3
Min Point:[-2.80432907  3.13001435]	 Min Value:8.663055331368261e-05
Learning Rate: 0.012936999084246104
epoch: 4
Min Point:[-2.80497677  3.13136565]	 Min Value:7.718328701956301e-07
Learning Rate: 0.014401807681089198
epoch: 5
Min Point:[-2.80511099  3.13130093]	 Min Value:6.924685461951561e-09
Learning Rate: 0.012946634315048874
epoch: 6
Min Point:[-2.80511682  3.131313  ]	 Min Value:6.217097887078336e-11
Learning Rate: 0.0144018076810892


In [34]:
def QuasiNewton(func,dfunc,hessian,Opti_func):
    '''
    func: target function
    Opti_func: Optimization function of Alpha
    '''

    # initialize a point
    x = np.array([-1,1])
    # calculated hessian matrix
    I=np.identity(x.size)
    H = I
    # set number of epochs
    epoch = 1000
    x_prev=x

    for i in range(epoch):
        #H=np.linalg.inv(H)
        gradient = dfunc(x)
        direction = -H*np.matrix(gradient).T
        direction = np.squeeze(np.asarray(direction))
#         #Wolfe condition line search
#         alpha = find_step_length(func, dfunc, x, 1, -dfx_next, c2=0.1)
        # Golden Search
        _,alpha,_=GoldenSectionSearchMethod(x,Opti_func,-direction,1e-3,1,1e-5)
#         # Backtracking line search 
#         while func(x + alpha*(-dfunc(x))) > func(x) + c*alpha*np.dot(dfunc(x).T, (-dfunc(x))):
#             print("alpha = %.4f | f(x + alpha*(-dfunc(x))) = %.4f" %
#                   (alpha, func(x + alpha*(-dfunc(x)))))
#             alpha = rho * alpha
        print("Learning Rate:",alpha)
        # set the new point
        x_prev=x
        x=x+np.dot(alpha,direction)
        #print(x.shape)
        if LA.norm(x-x_prev)<=1e-5 or LA.norm(x-x_prev)<=1e-5 or LA.norm(gradient)<=1e-5:
            break
        s=np.matrix(x-x_prev).T
        y=np.matrix(dfunc(x)-dfunc(x_prev)).T
        rho = float(1 / (y.T*s))
        H = (I - rho*s*y.T)*H*(I - rho*y*s.T) + rho*s*s.T

        print("epoch:",i)
        print("Min Point:"+str(x)+"\t Min Value:"+str(func(x)))

In [35]:
QuasiNewton(RosenbrockFunction,Derivative_Rosenbrock,Hessian_Rosenbrock,RosenbrockFunction_alpha)

Learning Rate: 0.5000005788572839
epoch: 0
Min Point:[1.00000232 1.        ]	 Min Value:2.1498510102125504e-09
Learning Rate: 0.0016652261593068257


In [36]:
QuasiNewton(HimmelblauFunction,Derivative_Himmelblau,Hessian_Himmelblau,HimmelblauFunction_alpha)

Learning Rate: 0.049522353412789585
epoch: 0
Min Point:[-2.08949178  3.27802826]	 Min Value:14.004969654491738
Learning Rate: 0.016080328972032258
epoch: 1
Min Point:[-2.75821227  3.38064846]	 Min Value:2.790944865380615
Learning Rate: 0.3264289896009511
epoch: 2
Min Point:[-2.82464553  3.13591738]	 Min Value:0.013206441328740295
Learning Rate: 0.6519288376716942
epoch: 3
Min Point:[-2.80501638  3.1317267 ]	 Min Value:7.291607562812035e-06
Learning Rate: 0.9999951823845986
epoch: 4
Min Point:[-2.80510677  3.13135617]	 Min Value:8.143365721085244e-08
Learning Rate: 0.9999951823845986
epoch: 5
Min Point:[-2.80511808  3.13131253]	 Min Value:3.742592505298837e-15
Learning Rate: 0.9999951823845986
